In [ ]:
from headers.directory_manager import *

ROOT_KAIST = "/home/dimitrios/THESIS_DATASETS/kaist_original"
BASELINE_MODEL = "./config/yolo11m.pt"
FINE_TUNED_IR_YOLO_MODEL = "./config/IR-YOLOv11m.pt"

nighttime_sequences = ["03", "04", "05", "10", "11"]


In [ ]:
flattened_nighttime_lwir_images = flatten_KAIST_images(sequences= nighttime_sequences, 
                                                       root_kaist= ROOT_KAIST, 
                                                       target_directory="preprocessed_nighttime", 
                                                       image_type= "lwir")

flattened_nighttime_rgb_images = flatten_KAIST_images(sequences= nighttime_sequences, 
                                                       root_kaist= ROOT_KAIST, 
                                                       target_directory="preprocessed_nighttime", 
                                                       image_type= "visible")

flattened_nighttime_labels = flatten_KAIST_labels(sequences= nighttime_sequences, 
                                                  root_kaist= ROOT_KAIST, 
                                                  target_directory="preprocessed_nighttime")

In [ ]:
flattened_nighttime_labels, flattened_nighttime_lwir_images, flattened_nighttime_rgb_images

In [ ]:
nighttime_pedestrian_and_empty_image_ids = "./config/nighttime_ped_emp_only.txt"

generate_txt_config_by_components(label_dir     = flattened_nighttime_labels,
                                strict          = True,
                                include_person  = True, 
                                include_people  = False, 
                                include_empty   = True, 
                                include_cyclist = False, 
                                output_file= nighttime_pedestrian_and_empty_image_ids)

In [ ]:
flattened_nighttime_labels = "/home/dimitrios/THESIS_DATASETS/kaist_original/preprocessed_nighttime/labels"
nighttime_pedestrian_and_empty_image_ids = "./config/nighttime_ped_emp_only.txt"


In [ ]:
cleaned_nighttime_labels_ped_emp = flattened_nighttime_labels + "ped_emp"

copy_files_based_on_config(input_dir=flattened_nighttime_labels, 
                           output_dir=cleaned_nighttime_labels_ped_emp, 
                           config_file=nighttime_pedestrian_and_empty_image_ids)

yolo_nighttime_labels = convert_kaist_to_yolo_labels(source_directory= cleaned_nighttime_labels_ped_emp)

In [ ]:
cleaned_nighttime_rgb_ped_emp = flattened_nighttime_rgb_images + "ped_emp"

copy_files_based_on_config(input_dir=flattened_nighttime_rgb_images, 
                           output_dir=cleaned_nighttime_rgb_ped_emp, 
                           config_file=nighttime_pedestrian_and_empty_image_ids)

In [ ]:
cleaned_nighttime_lwir_ped_emp = flattened_nighttime_lwir_images + "ped_emp"

copy_files_based_on_config(input_dir=flattened_nighttime_lwir_images, 
                           output_dir=cleaned_nighttime_lwir_ped_emp, 
                           config_file=nighttime_pedestrian_and_empty_image_ids)

In [ ]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

In [ ]:
cleaned_nighttime_lwir_ped_emp='/home/dimitrios/THESIS_DATASETS/kaist_original/preprocessed_nighttime/images/lwirped_emp'

In [ ]:
model = YOLO(FINE_TUNED_IR_YOLO_MODEL)
lwir_generated_yolo_labels = generate_yolo_labels(model=model, image_dir=cleaned_nighttime_lwir_ped_emp)

# generated labels for nighttime section of the dataset using the fine tuned IR-YOLO model 

In [ ]:
lwir_generated_yolo_labels = '/home/dimitrios/THESIS_DATASETS/kaist_original/preprocessed_nighttime/images/lwirped_emp_generated_yolo_labels'

train:
    images = cleaned_nighttime_rgb_ped_emp
    labels = lwir_generated_yolo_labels

test: 
    images = os.path.join(ROOT_KAIST, "annotations-xml-new-sanitized/set09/V000")
    labels = os.path.join(ROOT_KAIST, "/images/set09/V000/visible")

<!-- /home/dimitrios/THESIS_DATASETS/kaist_original/annotations-xml-new-sanitized/set09/V000 -->

In [ ]:
cleaned_nighttime_rgb_ped_emp = "/home/dimitrios/THESIS_DATASETS/kaist_original/preprocessed_nighttime/images/visibleped_emp"
lwir_generated_yolo_labels = '/home/dimitrios/THESIS_DATASETS/kaist_original/preprocessed_nighttime/images/lwirped_emp_generated_yolo_labels'

In [ ]:
dataset_generated_labels = {
    "train":{
        "images": cleaned_nighttime_rgb_ped_emp,
        "labels": lwir_generated_yolo_labels,
        },
    "test":{
        "images": os.path.join(ROOT_KAIST, "annotations-xml-new-sanitized/set09/V000"),
        "labels": os.path.join(ROOT_KAIST, "images/set09/V000/visible")
    }
}

generated_labels_tts = os.path.join(ROOT_KAIST, "tts_generated_labels_ir-yolov11m")

organize_dataset(root_path=generated_labels_tts, dataset_dict=dataset_generated_labels)

In [ ]:
yaml_ir_yolo_training = {
    "train": os.path.join(generated_labels_tts, "train"),
    "val":   os.path.join(generated_labels_tts, "test"),
    "nc": 1,
    "names": ['person']  
}

yaml_daytime_labels = "./config/ir-yolov11m_tts.yaml"

write_yolo_yaml(config_dict= yaml_ir_yolo_training, output_path=yaml_daytime_labels)

In [ ]:
from ultralytics import YOLO
import subprocess
import torch

torch.cuda.empty_cache()
try:
    subprocess.check_output('nvidia-smi')
    print('Nvidia GPU detected!')
except Exception: # this command not being found can raise quite a few different errors depending on the configuration
    print('No Nvidia GPU in system!')
    

torch.cuda.is_available()

# training of a baseline model for low-light pedestrian detection with generated labels

In [ ]:
model = YOLO(BASELINE_MODEL)  
results = model.train(data=yaml_daytime_labels, epochs=10, device=0, batch=8)

# data preprocessing for nighttime KAIST labels

the process is the same as before with the key difference of having the tran labels be the ones provided by KAIST

In [ ]:
kaist_yolo_format_nighttime_labels = "/home/dimitrios/THESIS_DATASETS/kaist_original/preprocessed_nighttime/labelsped_emp_yolo_format"

In [ ]:
dataset_generated_labels = {
    "train":{
        "images": cleaned_nighttime_rgb_ped_emp,
        "labels": kaist_yolo_format_nighttime_labels,
        },
    "test":{
        "images": os.path.join(ROOT_KAIST, "annotations-xml-new-sanitized/set09/V000"),
        "labels": os.path.join(ROOT_KAIST, "images/set09/V000/visible")
    }
    
    
    
}

generated_labels_tts = os.path.join(ROOT_KAIST, "tts_KAIST_labels")

organize_dataset(root_path=generated_labels_tts, dataset_dict=dataset_generated_labels)

In [ ]:
yaml_kaist_training = {
    "train": os.path.join(generated_labels_tts, "train"),
    "val":   os.path.join(generated_labels_tts, "test"),
    "nc": 1,
    "names": ['person']  
}

yaml_nighttime_labels_kaist = "./config/kaist_labels_tts.yaml"

write_yolo_yaml(config_dict= yaml_kaist_training, output_path=yaml_nighttime_labels_kaist)

In [ ]:
model = YOLO(BASELINE_MODEL)  
results = model.train(data=yaml_nighttime_labels_kaist, epochs=10, device=0, batch=8)